In [1]:
%%capture
!pip3 install matplotlib
!pip3 install pandas
!pip3 install py2neo
!pip3 install seaborn

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from IPython.display import display
from py2neo import Graph
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
host = 'localhost'
port = 7687
user = ''
password = ''

In [5]:
graph = Graph(
    host=host,
    port=port,
    user=user,
    password=password
)
tx = graph.begin()

In [6]:
with open('data/rfm.cql', 'r') as f:
    content = f.read()
    rfm_query = ' '.join(content.split())

In [7]:
results = tx.run(rfm_query).data()
df = pd.DataFrame(results)

Since the data are from 2011, the *recency* values seem quite high. That's not an important point, but to edit these values down I set the minimum value to zero.

In [8]:
df['recency'] = df['recency'] - df['recency'].min()
df.head()

,customer,recency,frequency,monetary
0,17850,302,35,5288.63
1,13047,31,18,3079.10
2,12583,2,18,7187.34
3,13748,95,5,948.25
4,15100,330,6,635.10


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
customer,4372.0,15299.677722,1722.390705,12346.00,13812.7500,15300.500,16778.250,18287.00
recency,4372.0,91.581199,100.772139,0.00,16.0000,50.000,143.000,373.00
frequency,4372.0,5.075480,9.338754,1.00,1.0000,3.000,5.000,248.00
monetary,4372.0,1898.459701,8219.345141,-4287.63,293.3625,648.075,1611.725,279489.02


I set 3 quantiles for *recency*, *frequency* and *monetary* values. Keep in mind, the value 3 here can change. But if you give a value greater than 3 for the *frequency*, you'll get an error like that,

```
ValueError: Bin edges must be unique: ...
You can drop duplicate edges by setting the 'duplicates' kwarg
```

In [10]:
df['r_val'] = pd.qcut(df['recency'], q=3, labels=range(3, 0, -1))
df['f_val'] = pd.qcut(df['frequency'], q=3, labels=range(1, 4))
df['m_val'] = pd.qcut(df['monetary'], q=3, labels=range(1, 4))

In [11]:
df['rfm_val'] = (
    df['r_val'].astype(str) + 
    df['f_val'].astype(str) + 
    df['m_val'].astype(str)
)

In [12]:
columns = [
    'recency',
    'frequency',
    'monetary'
]

mapping = {
    'Best customers': '333',
    'No purchases recently': '133',
    'Low loyalty': '111',
    'New customers': '311'
}

for k, v in mapping.items():
    print(k + ',')
    display(df[df.rfm_val == v][columns].describe().T)
    print()

Best customers,


,count,mean,std,min,25%,50%,75%,max
recency,725.0,8.616552,6.900977,0.00,3.00,8.00,14.00,24.00
frequency,725.0,15.612414,18.659721,5.00,7.00,11.00,17.00,248.00
monetary,725.0,7057.815131,19078.757582,1146.96,1959.38,3147.31,5452.17,279489.02



No purchases recently,


,count,mean,std,min,25%,50%,75%,max
recency,70.0,145.442857,56.633742,91.00,103.75,126.00,169.00,330.00
frequency,70.0,7.728571,4.235534,5.00,5.00,6.00,9.00,35.00
monetary,70.0,2703.509000,1693.392614,1143.27,1564.56,2097.25,3327.58,10217.48



Low loyalty,


,count,mean,std,min,25%,50%,75%,max
recency,746.0,239.353887,82.160120,91.00,172.0000,241.000,304.0000,373.00
frequency,746.0,1.215818,0.411664,1.00,1.0000,1.000,1.0000,2.00
monetary,746.0,175.151206,220.638499,-4287.63,115.6875,179.635,276.5175,374.57



New customers,


,count,mean,std,min,25%,50%,75%,max
recency,207.0,13.275362,6.999759,1.00,8.000,14.0,19.000,24.00
frequency,207.0,1.429952,0.496269,1.00,1.000,1.0,2.000,2.00
monetary,207.0,216.780048,93.770938,-17.45,159.105,216.3,298.915,373.75


| RFM Value | Description                                                         | Action                                                                                         |
|:----------|:--------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------|
| 333       | Best customers. <br>Most recently, most often, spend the most.      | Keep them.                                                                                     |
| 133       | Purchased frequently before, spent much. <br>No purchases recently. | Make special offers for them to come back.                                                     |
| 111       | Not recently, rarely, spend the little.                             | Low loyalty.<br>Try to increase loyalty. <br>For instance, give bonus points per transaction.  |
| 311       | Most recently, low frequency, spend the little.                     | Probably new customers.<br>Make a satisfaction survey.                                         |
| ...       | ...                                                                 | ...                                                                                            |